In [25]:
import pandas as pd
import csv
import numpy as np
from sklearn.linear_model import Ridge

# 1. Formula to best wine
### Attempt of finding a model that relates multiple factors of wine and quality (subjective).

In [26]:
# file read into 2D list then convert to numpy matrix
database = []
dataheader = 0
DATA_SIZE = 5000

with open('winequality-red.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    dataheader = spamreader.__next__()
    counter = 0
    for row in spamreader:
        row = np.asarray(row).astype(np.float64)
        database.append(row)
        counter += 1
        if counter > DATA_SIZE:
            break
# print(np.matrix(database))
database = np.matrix(database)

In [29]:
# divide into test and train data
# Then do ridge regression
test_size = database.shape[0] // 8
train_size = database.shape[0] - test_size
train_data, test_data = np.vsplit(database, np.array([train_size]))

train_x, train_y = np.hsplit(train_data, np.array([len(dataheader)-1]))
test_x, test_y = np.hsplit(test_data, np.array([len(dataheader)-1]))

rd = Ridge().fit(train_x, train_y)

In [31]:
# Display score
print(rd.score(train_x, train_y))
# print(lr.score(test_x, test_y))

# 
# Currently, score (R2) is really low
# This is probably due to not having enough data size compared to having lots of columns
# To get more meaningful 
# 

0.36865507887691423


# 2. Identify good wine
### By using same quality, classify wine into two categories: normal, and good(quality score 7 or above).

In [88]:
x = np.array([[1],[2],[3]])
# x = x.flatten()
x.ravel()


array([1, 2, 3])

In [ ]:
print(database)
#refer to the website below
#https://stackabuse.com/classification-in-python-with-scikit-learn-and-pandas/
import pandas as pd
import os
import sklearn as sk
from sklearn.neural_network import MLPClassifier


y = database[:,-1]
#converting the shape of y into 1d. 
#can you find a shorter way to convert it? Need to make (size,) not (1,size)
y = np.array(y)
y = y.ravel()

X = database[:,:-1]
print(X)
# print(X.shape, y.shape)
NN = MLPClassifier(solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (100,100), random_state = 1, max_iter=1000)
NN.fit(X,y)
print(NN.predict(X[1590:,:]))
round(NN.score(X,y), 4)
#the predicted quality scores don't seem to match with the original data.
#if you increase hidden_layer_sizes, the score incresase also, which makes sense
#but still, the last 10 predictions always show all 6's.
#edit: manipulate max_iter and get hight score(more precise prediction)

[[ 7.4    0.7    0.    ...  0.56   9.4    5.   ]
 [ 7.8    0.88   0.    ...  0.68   9.8    5.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    5.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     6.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    5.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     6.   ]]
[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]


In [13]:
# 
# use binary classification
# 

#this one is crap
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = read_csv("winequality-red.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
